In [4]:
print("\nImports...")
import sys
sys.path.append('/home/ebutz/ESL2024/code/utils' )
from play_with_complex import *
from data_utils import *
from train_utils import *
from model_utils import *
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import optuna
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler import NegativeSampling
import torch_geometric.transforms as T

import pickle
import os
import sys


import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

from nxontology.imports import from_file


Imports...


In [3]:
import wandb

config = {'homogeneous': False, 'scorelist_size': 1000, 'split_ratio': 0.8,
          'val_ratio': 0.1, 'test_ratio': 0.1, 'num_neighbors': [70, 55, 13, 89, 85],
          'batch_size': 1024, 'train_neg_sampling_ratio': 224, 'epochs': 18, 'disjoint_train_ratio': 0.6,
          'lr': 0.0015308253347932983, 'stopper_metric': 'mrr', 'stopper_direction': 'maximize',
          'stopper_patience': 5, 'stopper_frequency': 1, 'stopper_relative_delta': 0.05, 'gamma': 1.3,
          'alpha': 0.42680473078813763, 'gnn_layer': 'ResGatedGraphConv', 'dropout': 0.1,
          'norm': 'DiffGroupNorm', 'aggregation': 'min', 'hidden_channels': 115, 'num_layers': 3, 'attention_heads': 4}

run = wandb.init(config = config)

config = wandb.config

config.homogeneous = False

config.labels = {'head' : 'genes', 'relation' : 'gene_ontology', 'tail' : 'go'}

print(type(wandb.config))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: butzelliot (esl2024). Use `wandb login --relogin` to force relogin


<class 'wandb.sdk.wandb_config.Config'>


# Loading datas

In [7]:
# Iric
iric_path = '/home/ebutz/ESL2024/data/full_iric/iric.csv'
mapped_iric_path  = '/home/ebutz/ESL2024/data/full_iric/altailed_mapped_iric.pickle'
altails_dict_path = '/home/ebutz/ESL2024/data/full_iric/altail_iric_DICT.pickle'
check_dicts = True

data = load_iric_data(iric_path, featureless=False)
data  = T.ToUndirected(merge=True)(data) # Convert the graph to an undirected graph. Creates reverse edges for each edge.
data = T.RemoveDuplicatedEdges()(data) # Remove duplicated edges
print(data)
assert data.validate()

train_data, val_data, test_data = split_data(data, config)
train_loader, val_loader, test_loader = build_dataloaders(train_data, val_data, test_data, config)


FileNotFoundError: [Errno 2] No such file or directory: 'data/full_iric/iric.csv'

In [ ]:



# Model to train :
hidden_channels = 176
batch_size      = 4096
epochs          = 200
eval_period     = 2
lin_factor      = 1

use_wandb  = True

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_iric"
model_parameters_path = "/home/ebutz/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# Ontology
ontology_path = "/home/ebutz/ESL2024/data/go-basic.json.gz"

# ------------- Cuda ------------- #

print("\nCuda check...")

device = 'cpu'

# ------------- Loading datas ------------- #

iric = load_iric_data('data/iric.csv', featureless=False)
display(iric)

print("\nLoading iric...")
mapped_iric = pd.read_pickle(mapped_iric_path)
print(mapped_iric.head())
print('mapped_alt_tails type :', type(mapped_iric.iloc[0]['mapped_alt_tails']))

GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
map_to_GO = {value: key for key, value in GO_to_map.items()}

if check_dicts:
    looks_ok: bool = True
    for i in tqdm(range(len(list(mapped_iric['object']))), desc = "Checking GO to MAP dict"):
        if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i] :
            looks_ok = False
    print('GO - Mapping dicts looks ok :', looks_ok)

with open(altails_dict_path, 'rb') as handle:
    mapped_alt_tails = pickle.load(handle)
print("Alternative tails dict (first key-value pair):", list(mapped_alt_tails.items())[0])

# ------------- Loading ontology ------------- #

print("\nLoading ontology...")
nxo = from_file(ontology_path)
nxo.freeze()
pwc.nxo = nxo

# ------------- Making global variables accessibles to pwc ------------- #

pwc.map_to_GO        = map_to_GO
pwc.mapped_alt_tails = mapped_alt_tails
pwc.device           = device

# ------------- Making datasets ------------- #

print("\nMaking datasets...")
# # Edges index :
# heads = list(mapped_iric['mapped_subject'])
# tails = list(mapped_iric['mapped_object'])
# edge_index = torch.tensor([heads,tails], dtype=torch.long)
# # edges attributes :
# edge_attributes = torch.tensor(mapped_iric['mapped_predicate'])


# iric_pyg = Data(
#                 num_nodes = len(set(mapped_iric['object']).union(set(mapped_iric['subject']))),
#                 edge_index = edge_index,
#                 edge_attr = edge_attributes
#                 )

hetero_iric = iric_pyg.to_heterogeneous()
hetero_iric  = T.ToUndirected(merge=True)(hetero_iric) # Convert the graph to an undirected graph. Creates reverse edges for each edge.
hetero_iric = T.RemoveDuplicatedEdges()(hetero_iric)
print(hetero_iric)

print("\nDataset looks valid :",hetero_iric.validate(raise_on_error=True))

print('All done.')


Cuda check...


NameError: name 'load_iric_data' is not defined

# Splitting Datas

In [ ]:
def split_data(data, config):
    """
    Split the data into training, validation, and test sets using a random link split.

    The split is performed on the edges of the graph, and the function ensures that the reverse edge types are split accordingly to avoid leakage.
    See the documentation for torch_geometric.loader.RandomLinkSplit for more information.

    Parameters
    ----------
    data : torch_geometric.data.HeteroData
        The data to be split. This should be a HeteroData object containing the preprocessed data.
    config : object
        An object containing the configuration parameters for the split. This should include the ratios for the validation and test sets, 
        the disjoint train ratio, and the neg_sampling_ratio.

    Returns
    -------
    train_data, val_data, test_data : tuple of torch_geometric.data.HeteroData
        A tuple containing the training, validation, and test data. Each of these is a HeteroData object.
    """
    if config.labels:
        labels = (config.labels['head'], config.labels['relation'], config.labels['tail'])
        split = T.RandomLinkSplit(
            num_val= config.val_ratio,
            num_test= config.test_ratio,
            is_undirected=True,
            disjoint_train_ratio=config.disjoint_train_ratio, # Amount of supervision edges in train set
            neg_sampling_ratio=0,
            add_negative_train_samples=False, # If True, neg_sampling_ratio random negative samples per positive sample to the training set
            edge_types=[labels], # Only use these edge types for splitting
            rev_edge_types=[(config.labels['tail'], "rev_" + config.labels['relation'], config.labels['head'])], # Ensure that the reverse edge types are split accordingly to avoid leakage
        )
    else:
        split = T.RandomLinkSplit(
            num_val= config.val_ratio,
            num_test= config.test_ratio,
            is_undirected=True,
            disjoint_train_ratio=config.disjoint_train_ratio, # Amount of supervision edges in train set
            neg_sampling_ratio=0,
            add_negative_train_samples=False, # If True, neg_sampling_ratio random negative samples per positive sample to the training set
        )


    return split(data)

train_set, val_set, test_set = split_data(hetero_iric, config)

AttributeError: 'EdgeStorage' object has no attribute 'edge_index'

# Making Loader

In [ ]:
def build_dataloaders(train_data, val_data, test_data, config):
    """
    Build data loaders for training, validation, and testing.

    This function creates data loaders for the training, validation, and test sets. 
    The data loaders are used to load the data in batches during training and evaluation.
    Note that the loaders create their own negative samples.
    The validation and test loaders are created with a fixed number of negative samples, according to OGB's evaluation protocol.

    Parameters
    ----------
    train_data : torch_geometric.data.HeteroData
        The training data.
    val_data : torch_geometric.data.HeteroData
        The validation data.
    test_data : torch_geometric.data.HeteroData
        The test data.
    config : object
        An object containing the configuration parameters for the data loaders. This should include the number of neighbors to sample, 
        the negative sampling ratio, and the batch size.

    Returns
    -------
    train_loader, val_loader, test_loader : tuple of torch_geometric.loader.LinkNeighborLoader
        A tuple containing the data loaders for the training, validation, and test sets.
    """
    if config.labels:
        edge_label_index = ((config.labels['head'], config.labels['relation'], config.labels['tail']), train_data[config.labels['head'], config.labels['relation'], config.labels['tail']].edge_label_index)
    else:
        edge_label_index = None
        
    train_loader = LinkNeighborLoader(
        data=train_data,
        num_neighbors=config.num_neighbors, # Sample for each edge 20 neighbors in the first hop and at most 10 in the second.
        neg_sampling=NegativeSampling(mode="triplet", amount=config.train_neg_sampling_ratio), # Samples n negative edges per positive edge in the subgraph
        # Seed supervision edges from which to sample the subgraph that will be used for message passing.
        edge_label_index=edge_label_index,
        edge_label=None,
        batch_size=config.batch_size,
        shuffle=True, # Shuffle data each epoch
        pin_memory=True,
        num_workers=5
    )

    eval_batch_size = config.batch_size
    if config.train_neg_sampling_ratio < 1000: # Possible OOM issues as evaluation batch size are going to be larger than during training
        # Dynamically adapt batch size. This will cause evaluation to be slower, but it will prevent OOM errors.
        eval_batch_size = int(config.batch_size * (config.train_neg_sampling_ratio / 1000))

    val_loader = LinkNeighborLoader(
        data=val_data,
        num_neighbors=config.num_neighbors,
        neg_sampling=NegativeSampling(mode="triplet", amount=config.scorelist_size), # Samples 1000 negative edges per positive edge in the subgraph
        # Seed supervision edges from which to sample the subgraph that will be used for message passing.
        edge_label_index=edge_label_index,
        edge_label=None, # edge_label needs to be None for "triplet" negative sampling mode
        batch_size=eval_batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=5
    )

    test_loader = LinkNeighborLoader(
        data=test_data,
        num_neighbors=config.num_neighbors,
        neg_sampling=NegativeSampling(mode="triplet", amount=config.scorelist_size), # Samples 500 negative edges per positive edge in the subgraph
        # Seed supervision edges from which to sample the subgraph that will be used for message passing.
        edge_label_index=edge_label_index,
        edge_label=None, # edge_label needs to be None for "triplet" negative sampling mode
        batch_size=eval_batch_size,
        shuffle=True,
        pin_memory=True,
        num_workers=5
    )

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = build_dataloaders(train_data = train_set,
                                                          val_data   = val_set,
                                                          test_data  = test_set,
                                                          config     = config)

NameError: name 'config' is not defined

# Init model

In [ ]:
from torch_geometric.nn import *
from torch_geometric.nn.conv import *
from torch_geometric.nn.norm import *
import torch
from copy import deepcopy

class HeteroGNN(torch.nn.Module):
    """
    A Heterogeneous Graph Neural Network (HeteroGNN) class that applies multiple layers of a specified GNN layer to node features.

    Parameters
    ----------
    num_layers : int
        The number of GNN layers to apply.
    gnn_layer : torch.nn.Module
        The GNN layer to apply. This layer should be capable of handling bipartite graphs with differently sized node features.
    norm : callable
        A normalization function to apply to the node features after each GNN layer. If None, no normalization is applied.

    Attributes
    ----------
    convs : torch.nn.ModuleList
        A list of GNN layers to sequentially apply.
    norm : callable
        The normalization function to apply to the node features.

    Methods
    -------
    forward(x_dict, edge_index_dict):
        Apply the GNN layers to the node features.

    """
    def __init__(self, num_layers, gnn_layer, norm=None):
        super().__init__()

        # When working on bipartite graph with differently sized node features, layers need both lazy init and bipartite handling properties (see PyG GNN cheatsheet)
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(deepcopy(gnn_layer))
        self.norm = norm

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            if self.norm:
                x_dict = self.norm(x_dict)
            x_dict= x_dict.relu()
        return x_dict
    
class Linear(torch.nn.Module):
    """
    A class that applies a linear transformation to the node pair embeddings obtained from a GNN encoder for link prediction.

    Parameters
    ----------
    hidden_channels : int
        The size of the hidden layer (i.e the embedding size of nodes).
    dropout : float
        The dropout rate for the dropout layer.
    norm : callable
        A normalization function to apply to the node embeddings after the first linear transformation. If None, no normalization is applied.

    Attributes
    ----------
    lin1 : torch.nn.Linear
        The first linear transformation layer.
    lin2 : torch.nn.Linear
        The second linear transformation layer.
    dropout : torch.nn.Dropout
        The dropout layer.
    norm : callable
        The normalization function to apply to the node embeddings.

    Methods
    -------
    forward(z_dict, sampled_data):
        Apply the linear transformations to the node embeddings.

    """

    def __init__(self, config, norm=None):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * config.hidden_channels, config.hidden_channels)
        self.lin2 = torch.nn.Linear(config.hidden_channels, 1)
        self.dropout = torch.nn.Dropout(config.dropout)
        self.norm = norm
    def forward(self, z_dict, sampled_data, config):
        src_idx, dst_idx = get_sampled_edge_indices(sampled_data, config)

        x = torch.cat([z_dict[config.labels['tail']][src_idx], z_dict[config.labels['tail']][dst_idx]], dim=-1) # Concatenate node embeddings
        x = self.lin1(x)
        x = torch.relu(x)
        if self.norm:
            x = self.norm(x)
        x = self.dropout(x)
        x = self.lin2(x)
        return x.view(-1)

def get_gnn_layers(config):
    """
    Returns possible GNN layer architectures.
    
    Parameters
    ----------
    config : object
        An object containing the configuration parameters for the model. This should include the number of layers, the hidden dimension, 
        the number of attention heads (if required), the dropout rate, and the normalization layer.
    
    Returns
    -------
    layers : dict
        A dict containing the layer architectures.
    """
    
    if not hasattr(config, 'attention_heads'):
        config.attention_heads = 1

    gnn_layers = {
        "GATv2Conv" : GATv2Conv(in_channels= (-1,-1), out_channels=config.hidden_channels, heads=config.attention_heads, dropout=config.dropout, concat=False, add_self_loops=False),
        "TransformerConv" : TransformerConv(in_channels= (-1,-1), out_channels=config.hidden_channels, heads=config.attention_heads, concat=False, dropout=config.dropout),
        "MFConv" : MFConv(in_channels= (-1,-1), out_channels=config.hidden_channels, max_degree=10),
        "SAGEConv" : SAGEConv(in_channels= (-1,-1), out_channels=config.hidden_channels),
        "GraphConv" : GraphConv(in_channels= (-1,-1), out_channels=config.hidden_channels),
        "ResGatedGraphConv" : ResGatedGraphConv(in_channels= (-1,-1), out_channels=config.hidden_channels),
        "LEConv" : LEConv(in_channels= (-1,-1), out_channels=config.hidden_channels, aggr=config.aggregation),
        "GeneralConv" : GENConv(in_channels= (-1,-1), out_channels=config.hidden_channels, aggr=config.aggregation, learn_t=True, learn_p=True)
    }

    return gnn_layers

def get_norm_layers(config, nb_nodetypes):
    """
    Returns possible normalization layers.

    Parameters
    ----------
    config : object
        An object containing the configuration parameters for the model. This should include the hidden dimension, 
        alongside the number of node types in the graph.
    
    Returns
    -------
    norm_layers : dict
        A dict containing the normalization layers.
    """


    # Normalization layers pool
    norm_layers = {
        "BatchNorm" : BatchNorm(config.hidden_channels),
        "LayerNorm" : LayerNorm(config.hidden_channels),
        "InstanceNorm" : InstanceNorm(config.hidden_channels),
        "GraphNorm" : GraphNorm(config.hidden_channels),
        "GraphSizeNorm" : GraphSizeNorm(),
        "MeanSubtractionNorm" : MeanSubtractionNorm(),
        "DiffGroupNorm" : DiffGroupNorm(config.hidden_channels, groups=nb_nodetypes),
        "None" : None
    }

    return norm_layers

In [ ]:
class Model(torch.nn.Module):
    """
    A Model class that combines an encoder and a decoder for processing graph data.

    Parameters
    ----------
    config : object
        A configuration object that contains the parameters for the model. It should have the following attributes:
        - num_layers: The number of layers in the encoder.
        - gnn_layer: The index of the GNN layer to use in the encoder.
        - norm: The index of the normalization function to use in the encoder and decoder.
        - aggregation: The aggregation method to use in the encoder. It should be one of "sum", "mean", "min", "max", or "mul".
        - hidden_channels: The size of the hidden layer in the decoder.
        - dropout: The dropout rate for the decoder.
    norm_layers : dict of callable
        A list of normalization functions. The function to use is selected by the 'norm' attribute of the config object.
    gnn_layers : dict of torch.nn.Module
        A list of GNN layers. The layer to use is selected by the 'gnn_layer' attribute of the config object.

    Attributes
    ----------
    encoder : HeteroGNN
        The encoder part of the model.
    decoder : Linear
        The decoder part of the model.

    Methods
    -------
    forward(sampled_data):
        Apply the encoder and decoder to the sampled data.

    """

    def __init__(self, config, data, norm_layers, gnn_layers):
        super().__init__()
        self.encoder = HeteroGNN(num_layers=config.num_layers, gnn_layer=gnn_layers[config.gnn_layer], norm=norm_layers[config.norm])
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr=config.aggregation)
        self.decoder = Linear(config, norm=norm_layers[config.norm])

    def forward(self, sampled_data, config):
        z_dict = self.encoder(sampled_data.x_dict, sampled_data.edge_index_dict)
        return self.decoder(z_dict, sampled_data, config)

In [ ]:
data = hetero_iric

gnn_layers = get_gnn_layers(config)
norm_layers = get_norm_layers(config, len(data.node_types))

model = Model(config, data, norm_layers, gnn_layers).to(device)

# Test and train